# A/B Test: A New Menu Launch
## Project Overview
You're a business analyst for Round Roasters, a coffee restaurant in the United States of America. The executive team conducted a market test with a new menu and needs to figure whether the new menu can drive enough sales to offset the cost of marketing the new menu. Your job is to analyze the A/B test and write up a recommendation to whether the Round Roasters chain should launch this new menu.

In [1]:
# Load package
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_rel
from sklearn.neighbors import KDTree
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from statsmodels.tsa.seasonal import seasonal_decompose

import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = [12, 12]

## Step 1: Plan Your Analysis
To perform the correct analysis, you will need to prepare a data set. Prior to rolling up your sleeves and preparing the data, it’s a good idea to have a plan of what you need to do in order to prepare the correct data set. A good plan will help you with your analysis. Here are a few questions to get you started:

-What is the performance metric you’ll use to evaluate the results of your test?  
-What is the test period?  
-At what level (day, week, month, etc.) should the data be aggregated?  

In [2]:
# load Stores data
stores_data = pd.read_csv('round-roaster-stores.csv')
stores_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   StoreID                  133 non-null    int64  
 1   Sq_Ft                    133 non-null    int64  
 2   AvgMonthSales            133 non-null    int64  
 3   Right_Name               133 non-null    object 
 4   Phone Number             130 non-null    object 
 5   Street Combined          133 non-null    object 
 6   Street 1                 133 non-null    object 
 7   Street 2                 37 non-null     object 
 8   Street 3                 25 non-null     object 
 9   City                     133 non-null    object 
 10  State                    133 non-null    object 
 11  Postal Code              133 non-null    int64  
 12  Region                   133 non-null    object 
 13  Country                  133 non-null    object 
 14  Coordinates              1

In [3]:
stores_data.head(3)

,StoreID,Sq_Ft,AvgMonthSales,Right_Name,Phone Number,Street Combined,Street 1,Street 2,Street 3,City,State,Postal Code,Region,Country,Coordinates,Latitude,Longitude,Timezone,Current Timezone Offset,Olson Timezone
0,10018,1183,18000,Bellflower & Spring,562-420-1317,"2890 N Bellflower Blvd, #A-1, The Los Altos Ma...",2890 N Bellflower Blvd,#A-1,The Los Altos Marketplace,Long Beach,CA,908151125,West,US,"(33.8085823059082, -118.124931335449)",33.808582,-118.124931,Pacific Standard Time,-480,GMT-08:00 America/Los_Angeles
1,10068,1198,16000,"Foothill & Boston, La Crescenta",818-541-7693,"3747 Foothill Boulevard, A",3747 Foothill Boulevard,A,NaN,La Cresenta,CA,912141700,West,US,"(34.2375450134277, -118.26114654541)",34.237545,-118.261146,Pacific Standard Time,-480,GMT-08:00 America/Los_Angeles
2,10118,1204,13000,Magic Mountain & Tourney,661-260-0844,"25349 Wayne Mills Place, Tourney Retail Plaza",25349 Wayne Mills Place,NaN,Tourney Retail Plaza,Valencia,CA,913551827,West,US,"(34.4237022399902, -118.579261779785)",34.423702,-118.579262,Pacific Standard Time,-480,GMT-08:00 America/Los_Angeles


In [4]:
# load Transactions data
# force Invoice Date column to datetime 
transactions_data = pd.read_csv('RoundRoastersTransactions.csv', parse_dates=['Invoice Date'])
transactions_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4332333 entries, 0 to 4332332
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   StoreID         int64         
 1   Invoice Number  int64         
 2   Invoice Date    datetime64[ns]
 3   SKU             int64         
 4   Category        object        
 5   Product         object        
 6   QTY             int64         
 7   Size            object        
 8   Gross Margin    float64       
 9   Sales           float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 330.5+ MB


In [5]:
transactions_data.head()

,StoreID,Invoice Number,Invoice Date,SKU,Category,Product,QTY,Size,Gross Margin,Sales
0,10018,16296643,2015-01-21,1043,Espresso,Mocha,3,L,6.7365,14.97
1,10018,16296643,2015-01-21,2001,Pastry,Croissant,1,NaN,1.1000,2.75
2,10018,16297717,2015-01-21,1021,Espresso,Espresso,3,S,4.1850,8.37
3,10018,16297717,2015-01-21,1022,Espresso,Espresso,4,M,5.9800,11.96
4,10018,16297717,2015-01-21,1023,Espresso,Espresso,3,L,4.7850,9.57


In [6]:
# load Treatment Stores data
treatment_stores_data = pd.read_csv('treatment-stores.csv')
treatment_stores_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   StoreID                  10 non-null     int64  
 1   Sq_Ft                    10 non-null     int64  
 2   AvgMonthSales            10 non-null     int64  
 3   Phone Number             9 non-null      object 
 4   Street Combined          10 non-null     object 
 5   Street 1                 10 non-null     object 
 6   Street 2                 1 non-null      float64
 7   Street 3                 1 non-null      object 
 8   City                     10 non-null     object 
 9   Postal Code              10 non-null     int64  
 10  Region                   10 non-null     object 
 11  Country                  10 non-null     object 
 12  Coordinates              10 non-null     object 
 13  Latitude                 10 non-null     float64
 14  Longitude                10 n

In [7]:
treatment_stores_data.head(3)

,StoreID,Sq_Ft,AvgMonthSales,Phone Number,Street Combined,Street 1,Street 2,Street 3,City,Postal Code,Region,Country,Coordinates,Latitude,Longitude,Timezone,Current Timezone Offset,Olson Timezone,Name,Right_State
0,1664,1475,11000,8478428048,"101 W. Main St., Barrington Village Center",101 W. Main St.,NaN,Barrington Village Center,Barrington,60010,Central,US,"(42.1540565490723, -88.1362915039063)",42.154057,-88.136291,Central Standard Time,-360,GMT-06:00 America/Chicago,Barrington,IL
1,1675,1472,15000,8472531188,90 East Northwest Highway,90 East Northwest Highway,NaN,NaN,Mount Prospect,60056,Central,US,"(42.0633544921875, -87.9355773925781)",42.063354,-87.935577,Central Standard Time,-360,GMT-06:00 America/Chicago,Northwest Hwy & Elmhurst Rd,IL
2,1696,1471,10000,2242232528,1261 East Higgins Road,1261 East Higgins Road,NaN,NaN,Schaumburg,60173,Central,US,"(42.039363861084, -88.048828125)",42.039364,-88.048828,Central Standard Time,-360,GMT-06:00 America/Chicago,Higgins & Meacham,IL


## Step 2: Clean Up Your Data
In this step, you should prepare the data for steps 3 and 4. You should aggregate the transaction data to the appropriate level and filter on the appropriate data ranges. You can assume that there is no missing, incomplete, duplicate, or dirty data. You’re ready to move on to the next step when you have weekly transaction data for all stores.

In [8]:
# Test cities: Denver and Chicago
# Treatment: 12 Weeks [2016-April-29 to 2016-July-21], start on Friday
# Control: 12 Weeks [(2015-April-29 to 2015-July-21], start on Wednesday
# Total weeks to identify trend and season: 52 Weeks + treatment weeks + control weeks = 52 + 12 + 12 = 76 Weeks

data_end_date = datetime(2016, 7, 21)
data_start_date = data_end_date - timedelta(weeks=76)
print(f'Data Start Date: {data_start_date} \nData End Date: {data_end_date}')

Data Start Date: 2015-02-05 00:00:00 
Data End Date: 2016-07-21 00:00:00


In [9]:
# Filter data for further process
_trans = transactions_data.query('`Invoice Date` > @data_start_date and `Invoice Date` <= @data_end_date')
# Make Invoice Date Index
_trans.set_index('Invoice Date', inplace=True)

# Aggregate the data to get the weekly gross margin and weekly traffic count unique invoices
agg_rules = {'Gross Margin': 'sum', 'Invoice Number': 'nunique'}
_trans = _trans.groupby([pd.Grouper(freq='W-FRI', closed='left'), 'StoreID']).agg(agg_rules).reset_index()
_trans.rename(columns={'Invoice Number': 'Weekly Foot Traffic'}, inplace=True)

# Hack to start week on first date
_trans['Invoice Date'] = _trans['Invoice Date'] - pd.offsets.Week(1)

# Create Trend and Seasonal
result = seasonal_decompose(_trans['Weekly Foot Traffic'], period=12, extrapolate_trend='freq')

# Add Trend and Seasonal weekly gross data
_trans = _trans.assign(Trend = result.trend, Seasonal = result.seasonal)

# join weekly transactions and stores data 
stores_columns = ['StoreID', 'Sq_Ft', 'AvgMonthSales', 'Region']
_trans = _trans.merge(stores_data[stores_columns], on='StoreID')

# Add group variables to merged data
_is_treatment = _trans['StoreID'].isin(treatment_stores_data['StoreID'])
weekly_gross_and_traffic = _trans.assign(Group = np.where(_is_treatment, 'Treatment', 'Control'))

# Test the progress
weekly_gross_and_traffic.query('StoreID == 10018 and `Invoice Date` == @datetime(2015, 2, 6)')

,Invoice Date,StoreID,Gross Margin,Weekly Foot Traffic,Trend,Seasonal,Sq_Ft,AvgMonthSales,Region,Group
7068,2015-02-06,10018,2212.7105,308,290.125,-0.078252,1183,18000,West,Control


In [10]:
# Filter Post and Pre test data
post_data = weekly_gross_and_traffic.query('`Invoice Date` >= @datetime(2016, 4, 29)')
pre_data = weekly_gross_and_traffic.query('`Invoice Date` >= @datetime(2015, 4, 29) and `Invoice Date` <= @datetime(2015, 7, 21)')

pre_data.head()

,Invoice Date,StoreID,Gross Margin,Weekly Foot Traffic,Trend,Seasonal,Sq_Ft,AvgMonthSales,Region,Group
12,2015-05-01,1508,1701.9960,230,227.291667,0.653029,1457,12000,Central,Control
13,2015-05-08,1508,1551.2640,216,220.958333,-2.668948,1457,12000,Central,Control
14,2015-05-15,1508,1551.1995,204,217.625000,-3.644147,1457,12000,Central,Control
15,2015-05-22,1508,1626.8105,228,202.000000,2.524783,1457,12000,Central,Control
16,2015-05-29,1508,937.5550,125,195.958333,2.459223,1457,12000,Central,Control


In [11]:
# Aggregate Pre and Post data per store
agg_store_rules = {
    'Gross Margin': 'sum', 
    'Weekly Foot Traffic': 'sum', 
    'Trend': 'sum', 
    'Seasonal': 'sum', 
    'Sq_Ft': 'first',
    'AvgMonthSales': 'first',
    'Region': 'first',
    'Group': 'first'
}

def agg_per_store(df, agg_rule = agg_store_rules):
    return df.groupby(['StoreID'], as_index=False).agg(agg_rule)

In [12]:
# Pre Data per store
pre_data_per_store = agg_per_store(pre_data)

# Post Data per store
post_data_per_store = agg_per_store(post_data)

In [13]:
post_data_per_store.head()

,StoreID,Gross Margin,Weekly Foot Traffic,Trend,Seasonal,Sq_Ft,AvgMonthSales,Region,Group
0,1508,19035.5010,2516,2635.708333,0.000000e+00,1457,12000,Central,Control
1,1542,20272.3985,2918,2711.875000,4.440892e-16,1506,12000,Central,Control
2,1580,18698.5225,2567,2800.458333,-5.551115e-16,1568,14000,Central,Control
3,1630,21627.3460,2797,2991.041667,6.661338e-16,1582,17000,Central,Control
4,1662,12714.1125,1703,3066.041667,1.110223e-16,1471,11000,Central,Control


## Step 3: Match Treatment and Control Units
In this step, you should create the trend and seasonality variables, and use them along with you other control variable(s) to match two control units to each treatment unit. Treatment stores should be matched to control stores in the same region. Note: Calculate the number of transactions per store per week and use 12 periods to calculate trend and seasonality.  

Apart from trend and seasonality...  

-What control variables should be considered? Note: Only consider variables in the RoundRoastersStore file.  
-What is the correlation between your each potential control variable and your performance metric? (Example of correlation matrix below)  
-What control variables will you use to match treatment and control stores?

In [14]:
# Correlation Gross Margin and Stores variables
pre_data_per_store[['Gross Margin', 'Sq_Ft', 'AvgMonthSales']].corr().round(2)


,Gross Margin,Sq_Ft,AvgMonthSales
Gross Margin,1.00,-0.04,0.98
Sq_Ft,-0.04,1.00,-0.05
AvgMonthSales,0.98,-0.05,1.00


In [15]:
selected_variables = ['Trend', 'Seasonal', 'AvgMonthSales']
selected_variables

['Trend', 'Seasonal', 'AvgMonthSales']

## Step 4: Analysis and Writeup
Conduct your A/B analysis and create a short report outlining your results and recommendations.  

In an AB Analysis we use the correlation matrix to find the most correlated variable to the performance metric to include in the AB controls tool to help find the best matches.

In [16]:
# Match Pre Treatment and Control Stores Data
pre_control_stores = pre_data_per_store.query('Group == "Control"').reset_index(drop=True)
pre_treatment_stores = pre_data_per_store.query('Group == "Treatment"').reset_index(drop=True)

# filter region data
# Region: 'Central'
pre_control_central = pre_control_stores.query('Region == "Central"').reset_index(drop=True)
pre_treatment_central = pre_treatment_stores.query('Region == "Central"').reset_index(drop=True)

# Region: 'West'
pre_control_west = pre_control_stores.query('Region == "West"').reset_index(drop=True)
pre_treatment_west = pre_treatment_stores.query('Region == "West"').reset_index(drop=True)


In [17]:
# Scale Data
transformer = ColumnTransformer([('scaler', StandardScaler(), selected_variables)], remainder='drop')
transformer.fit(pre_control_stores)

ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                 ['Trend', 'Seasonal', 'AvgMonthSales'])])

In [18]:
# Matched Treatment and Stores - version: 2
# return: 'Treatment store ID', 'Control store ID', 'Pre Gross Treatment', 'Pre Gross Control'

def match_treatment_control_stores(region_control_data, region_treatment_data):
    treatment_ids = []
    control_ids = []
    treatment_store_gross = []
    control_store_gross = []

    kdtree = KDTree(transformer.transform(region_control_data), leaf_size=2)

    for i , treat_store_id in enumerate(region_treatment_data['StoreID']):
        treatment_ids += [treat_store_id, treat_store_id]
        treat_data = region_treatment_data.iloc[i:i+1]
        
        _tg = treat_data.iloc[0]['Gross Margin'] # Get column value from df with single row
        treatment_store_gross += [_tg,_tg]
        
        idx = kdtree.query(transformer.transform(treat_data), k=2, return_distance=False, dualtree=True)

        _matched = region_control_data.iloc[idx[0]]
        control_ids += list(_matched['StoreID'])
        control_store_gross += list(_matched['Gross Margin'])
    
    _x = list(zip(treatment_ids, control_ids, treatment_store_gross, control_store_gross))
    _columns = ['Treatment StoreID', 'Control StoreID', 'Treatment Pre Gross', 'Control Pre Gross']
    return pd.DataFrame(_x, columns=_columns)

In [19]:
# Find Matched Stores for Central and West Region
stores_matched_west = match_treatment_control_stores(pre_control_west, pre_treatment_west)
stores_matched_central = match_treatment_control_stores(pre_control_central, pre_treatment_central)


In [20]:
# Join Post Data to region region matched stores

def pre_post_data_report(matched_stores, post_stores_data):
    post_column = ['StoreID', 'Gross Margin']
    _psd = post_stores_data[post_column].set_index('StoreID')

    # treatment
    _res = matched_stores.join(_psd, on='Treatment StoreID').rename(columns={'Gross Margin': 'Treatment Post Gross'})
    # control
    _res = _res.join(_psd, on='Control StoreID').rename(columns={'Gross Margin': 'Control Post Gross'})

    # treatment changes
    _chg_treat = _res[['Treatment Pre Gross', 'Treatment Post Gross']].pct_change(axis=1).dropna(axis=1)
    _res['Treatment Change'] = _chg_treat * 100 # make it percentage

    # control changes
    _chg_control = _res[['Control Pre Gross', 'Control Post Gross']].pct_change(axis=1).dropna(axis=1)
    _res['Control Change'] = _chg_control * 100 # make it percentage

    return _res


In [24]:
# Region Reports
central_report = pre_post_data_report(stores_matched_central, post_data_per_store)
west_report = pre_post_data_report(stores_matched_west, post_data_per_store)

west_report

,Treatment StoreID,Control StoreID,Treatment Pre Gross,Control Pre Gross,Treatment Post Gross,Control Post Gross
0,2288,8917,17778.9450,19555.9445,27610.1000,16618.8065
1,2288,12269,17778.9450,24016.0060,27610.1000,24390.8225
2,2293,12286,14208.5030,14675.5610,18782.6300,13028.7550
3,2293,9081,14208.5030,16226.1305,18782.6300,19060.4205
4,2301,12169,15555.4480,16277.4935,17474.3110,14382.9335
5,2301,12019,15555.4480,17926.7355,17474.3110,17359.4295
6,2322,9438,19012.3295,19310.7850,27599.1955,24261.8455
7,2322,2468,19012.3295,19018.5430,27599.1955,17393.4290
8,2341,9524,15639.3250,15719.5550,23589.7275,15856.7770
9,2341,9188,15639.3250,12891.9535,23589.7275,13075.8970


In [32]:
# Test if the Pre and Post Gross is not random
# West Region Test
s_w, p_w = ttest_ind(west_report['Treatment Pre Gross'], west_report['Treatment Post Gross'])
print('West Region Test', '\nStatistic:', s_w, 'P-Value:', p_w, 'Confidence: ', (1-p_w)*100)

# Central Region Test
s_c, p_c = ttest_ind(central_report['Treatment Pre Gross'], central_report['Treatment Post Gross'])
print('Central Region Test', '\nStatistic:', s_c, 'P-Value:', p_c, 'Confidence: ', (1-p_c)*100)


West Region Test 
Statistic: -4.284551362855903 P-Value: 0.00044612063836174627 Confidence:  99.95538793616383
Central Region Test 
Statistic: -2.733830994627133 P-Value: 0.013634341572891347 Confidence:  98.63656584271087


In [47]:
# TODO
# compare the changes from pre and post between treatment and control


,Treatment StoreID,Control StoreID,Treatment Pre Gross,Control Pre Gross,Treatment Post Gross,Control Post Gross,Treatment_Change
0,1664,7334,18086.9585,14875.4745,21720.7275,12965.8825,0.200905
1,1664,8562,18086.9585,12543.8040,21720.7275,11785.5585,0.200905
2,1675,2014,21620.2310,23389.3185,31738.2480,19605.0480,0.467988
3,1675,1630,21620.2310,23771.1470,31738.2480,21627.3460,0.467988
4,1696,7334,15943.8625,14875.4745,21168.7500,12965.8825,0.327705
5,1696,8562,15943.8625,12543.8040,21168.7500,11785.5585,0.327705
6,1700,7484,23735.7890,21794.7185,31217.9890,22840.7860,0.315229
7,1700,1857,23735.7890,21911.6010,31217.9890,23649.7920,0.315229
8,1712,8312,27426.7110,27264.9955,45944.8240,24642.2860,0.675185
9,1712,7434,27426.7110,21174.1440,45944.8240,21589.1325,0.675185
